## EVs price estimation model

Installing environment.yml by opening a terminal and typing

conda env create --name NAME_OF_YOUR_ENV --file=environment.yml

then activate the environment by typing 

conda activate NAME_OF_YOUR_ENV

- - -

### Importing dependencies and loading input yml files
Start by parsing input yml files and loading dependencies on core.model classes

In [1]:
import pandas as pd
import yaml
import os

from core.model import EVs, Minerals, Batteries

input_ymls = ['paths', 'new_names', 'assumptions']
inputs = {}

# Load inputs, paths and assumptions from yml files
for input_yml in input_ymls:
    with open(f'inputs/{input_yml}.yml') as f:
        inputs[input_yml] = yaml.load(f, Loader=yaml.FullLoader)

We are now prodiding information on the sets of our model

In [2]:
models_list = ['HIST','GCAM']

minerals_list = ['Li2CO3']

chemistries_list = ['NMC','LFP']


---
### Vehicles
Initialize the EVs class by providing its assumptions and parsing production (i.e. sales) projected by each model 

In [3]:
vehicles = EVs(assumptions = inputs['assumptions']['vehicles'])

for model in models_list:
    path=inputs['paths']['data']['vehicles']['production'][model]
    vehicles.parse_production(path=path,model=model)

We need to parse the historical prices and to set the last historical year according to the data we found (2020 in this case)

In [4]:
vehicles.parse_historical_prices(path=inputs['paths']['data']['vehicles']['prices'])
vehicles.set_last_hist_year(2020)

We may also need to rename column values with more readable lables and convert passenger-km or ton-km to vehicles-km

In [5]:
vehicles.rename_column_values(new_names=inputs['new_names'])
vehicles.convert_km_to_vehicles(units_to_convert=['million ton-km','million pass-km'])

---
### Minerals
Initialize the Minerals class by providing its assumptions. For each mineral in the list above-provided, we need to parse historical price and production data to fit the linear regression model and estimate future prices

In [6]:
minerals = Minerals(assumptions = inputs['assumptions']['minerals'])

for mineral in minerals_list:
    path = inputs['paths']['data']['minerals'][mineral]
    minerals.parse_historical_price(path=path,mineral=mineral)
    minerals.parse_historical_production(path=path,mineral=mineral)

For each mineral we need to estimate the historical production required to manufacture EVs, and to estimate the future one, according the production of mineral not required for EVs will be the same as the average in the "last_valid_years" 

In [7]:
for mineral in minerals_list:
    minerals.get_production_for_ev(mineral=mineral,vehicle_production=vehicles.production)
    minerals.get_future_production(mineral=mineral,last_valid_years=3,last_hist_year=2020)

For each mineral, we need to estimate future prices with a linear regression model trained on historical correlation between price and production.

In [8]:
for mineral in minerals_list:
    minerals.estimate_future_prices(
            mineral=mineral,
            last_hist_year=2020,
            # Add parameters for the linear regression model
        )

---
### Batteries
Initialize the Batteries class with its own assumptions. For each chemistry, parse historical price and production (i.e. capacity) data and historical and future chemistry market shares data

In [9]:
batteries = Batteries(assumptions = inputs['assumptions']['batteries'])

for chemistry in chemistries_list:
    path = inputs['paths']['data']['batteries'][chemistry]
    batteries.parse_historical_price(path=path,chemistry=chemistry)
    batteries.parse_historical_production(path=path,chemistry=chemistry)
    batteries.parse_market_shares(path=os.path.join(inputs['paths']['data']['batteries']['market_shares']))  

For each chemistry, estimate future production based on global ev production 

In [10]:
for chemistry in chemistries_list:
    batteries.get_future_production(
            chemistry = chemistry,
            ev_sales = vehicles.get_global_production(excluded_cols=[]),
        )

Estimate future prices of batteries based on the logaritmic trend provided by Penisa et al. (https://doi.org/10.3390/en13205276). 

NB.1 This trend may be changed.

NB.2 This estimation also accounts for the impact on rising minerals prices


In [11]:
for chemistry in chemistries_list:    
    batteries.estimate_future_prices(
        chemistry = chemistry,
        minerals = minerals,
        mode = 'log',
    )

---
### Future vehicles prices estimation
All information to estimate future vehicles prices have been calculated. This will be calculated for the average vehicles in each market-share, by subsector, model, scenario, region and year

In [12]:
vehicles.estimate_future_prices(batteries=batteries)

---
### Results export
The results are ready to be exported

In [14]:
vehicles.prices.to_csv(inputs['paths']['results']['vehicles']['prices'],index=False)
minerals.prices.to_csv(inputs['paths']['results']['minerals']['prices'],index=False)
batteries.prices.to_csv(inputs['paths']['results']['batteries']['prices'],index=False)
vehicles.production.to_csv(inputs['paths']['results']['vehicles']['production'],index=False)
minerals.production.to_csv(inputs['paths']['results']['minerals']['production'],index=False)
batteries.production.to_csv(inputs['paths']['results']['batteries']['production'],index=False)